In [20]:
pip install xlrd

SyntaxError: invalid syntax (2466519365.py, line 1)

In [21]:
pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 8.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marce\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [23]:
import pandas as pd
import xlrd
import psycopg2
from sqlalchemy import create_engine

## Cleaning Data

In [15]:
file_path = 'dataset/ibge/PIB dos Municípios - base de dados 2010-2020.xls'
df = pd.read_excel(file_path)

# print the column names to verify their exact names in the dataset
print(df.columns)

Index(['Ano', 'Código da Grande Região', 'Nome da Grande Região',
       'Código da Unidade da Federação', 'Sigla da Unidade da Federação',
       'Nome da Unidade da Federação', 'Código do Município',
       'Nome do Município', 'Região Metropolitana', 'Código da Mesorregião',
       'Nome da Mesorregião', 'Código da Microrregião', 'Nome da Microrregião',
       'Código da Região Geográfica Imediata',
       'Nome da Região Geográfica Imediata',
       'Município da Região Geográfica Imediata',
       'Código da Região Geográfica Intermediária',
       'Nome da Região Geográfica Intermediária',
       'Município da Região Geográfica Intermediária',
       'Código Concentração Urbana', 'Nome Concentração Urbana',
       'Tipo Concentração Urbana', 'Código Arranjo Populacional',
       'Nome Arranjo Populacional', 'Hierarquia Urbana',
       'Hierarquia Urbana (principais categorias)', 'Código da Região Rural',
       'Nome da Região Rural',
       'Região rural (segundo classificação d

In [17]:
# select the desired columns with exact names
selected_columns = [
    'Ano',
    'Código da Grande Região',
    'Nome da Grande Região',
    'Código da Unidade da Federação',
    'Sigla da Unidade da Federação',
    'Nome da Unidade da Federação',
    'Código do Município',
    'Nome do Município',
   'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)',
       'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)',
]

# create a new DataFrame with the selected columns
filtered_df = df[selected_columns]

# rename the columns
filtered_df.columns = [
    'year',
    'region_code',
    'region_name',
    'uf_code',
    'uf_acronym',
    'uf_name',
    'municipality_code',
    'municipality_name',
    'gdp',
    'gdp_capita'
]

# Display the first few rows of the filtered DataFrame
filtered_df.head()

,year,region_code,region_name,uf_code,uf_acronym,uf_name,municipality_code,municipality_name,gdp,gdp_capita
0,2010,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,262076.878,10731.18
1,2010,1,Norte,11,RO,Rondônia,1100023,Ariquemes,1364693.780,15103.86
2,2010,1,Norte,11,RO,Rondônia,1100031,Cabixi,69611.114,11033.62
3,2010,1,Norte,11,RO,Rondônia,1100049,Cacoal,1186494.091,15095.15
4,2010,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,222021.189,13037.06


In [19]:
# check for null values in all rows
null_check = filtered_df.isnull().any(axis=1)

# check if there are any rows with null values
rows_with_null = filtered_df[null_check]

# display rows with null values (if any)
if not rows_with_null.empty:
    print("Rows with null values:")
    print(rows_with_null)
else:
    print("No rows with null values.")


No rows with null values.


## Uploading data to Postgres

In [31]:
# Set up a connection to your PostgreSQL RDS instance
host = 'database-2.ckuaogoaistw.us-east-1.rds.amazonaws.com'
port = 5432
database = 'postgres'
user = 'postgres'
password = 'postgres'

connection = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)

# create a cursor object
cursor = connection.cursor()

# create table
table_name = 'ibge_data' 

# Set up a SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Use Pandas to create the table and insert data
filtered_df.to_sql(
    table_name,
    engine,
    if_exists='replace',  # You can use 'replace', 'append', or 'fail' depending on your needs
    index=False  # Set to False if you don't want to include the DataFrame index as a column
)

# Commit changes and close the cursor and connection
connection.commit()
cursor.close()
connection.close()